In [2]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from google.colab import files

In [16]:
HEADERS = {'User-Agent':
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) \
        AppleWebKit/537.36 (KHTML, like Gecko) \
        Chrome/90.0.4430.212 Safari/537.36'}

COOKIES = {}

In [17]:
def get_data(url):
  page = requests.get(url, cookies=COOKIES, headers=HEADERS)
  return page

In [30]:
def fetch_href(soup):
  links = []
  for items in soup.findAll("a" , {'data-hook':"see-all-reviews-link-foot"}):
      links.append(items['href'])
  return links[0]

In [32]:
def customer_review(soup):
    data_str = ""
    td_tag_list = soup.findAll(lambda tag: "data-hook" in tag.attrs and tag["data-hook"] == "review")
    for item in td_tag_list:
        selected_item = item.findAll("span", attrs={'class': 'a-size-base review-text review-text-content'})
        if selected_item:
            temp_text = selected_item[0].get_text()
            if len(temp_text) > 0:
                data_str = data_str + temp_text
            else:
                data_str = data_str + " \n"
        else:
            data_str = data_str + " \n"
    result = data_str.split("\n")
    return (result)

In [33]:
def customer_rating(soup):
    data_out_list = []
    data_str = ""
    td_tag_list = soup.findAll(lambda tag: "data-hook" in tag.attrs and tag["data-hook"] == "review-star-rating")
    for item in td_tag_list:
        selected_item = item.findAll("span", attrs={'class': 'a-icon-alt'})
        if selected_item:
            data_str = selected_item[0].get_text().split("out")[0].strip(" ")
            data_int = int(float(data_str))
            data_out_list.append(data_int)
    return data_out_list

In [34]:
def extract_r_and_r(data_asin):
  all_reviews = []
  review_data = []
  rating_data = []
  url = f"https://www.amazon.in/dp/{data_asin}"
  
  response = get_data(url)
  soup = BeautifulSoup(response.content)
  link = fetch_href(soup)



  i = 0
  print(f"Fetching reviews from the product: {data_asin}")

  while 1:
    i += 1
    url = f"https://www.amazon.in{link}&pageNumber={i}"
    response = get_data(url)
    soup = BeautifulSoup(response.text)
    review_data = customer_review(soup)
    review_data = [review for review in review_data if len(review) > 0]
    temp_rating_data = customer_rating(soup)
    rating_data.extend(temp_rating_data)
    if len(review_data) == 0:
        break
    all_reviews += review_data
  reviews_df = pd.DataFrame({'reviews': all_reviews,'ratings': rating_data})
  reviews_df["ASIN"] = data_asin
  print("{} has {} reviews".format(data_asin, reviews_df.shape[0]))
  return reviews_df

In [37]:
data_asins = ['B09T4T4596' , 'B09QRTXCWD', 'B09QSBPKTF', 'B09QRL1M8Y', 'B09QRQQL51']
#data_asins = ['B09T4T4596']
r_and_r_file_name = "Reviews_and_Ratings.xlsx"


li = []
for asins in data_asins:
    df = extract_r_and_r(asins)
    li.append(df)
reviews_df = pd.concat(li, axis=0,ignore_index=True)

Fetching reviews from the product: B09T4T4596
B09T4T4596 has 7 reviews
Fetching reviews from the product: B09QRTXCWD
B09QRTXCWD has 59 reviews
Fetching reviews from the product: B09QSBPKTF
B09QSBPKTF has 59 reviews
Fetching reviews from the product: B09QRL1M8Y
B09QRL1M8Y has 6 reviews
Fetching reviews from the product: B09QRQQL51
B09QRQQL51 has 8 reviews


In [38]:
reviews_df.to_excel(r_and_r_file_name, index=False)

In [ ]:
files.download(r_and_r_file_name) 